# Project 3 - Homework group 19 Abhinav, Mukul,Vijaya


## Ensuring correct html export

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Importing Libraries


### Installing dependencies

In [2]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install igraph

Note: you may need to restart the kernel to use updated packages.


### Imports

In [4]:
import pandas as pd
import numpy as np


from ts2vg import NaturalVG
from ts2vg import HorizontalVG



import plotly.offline as py
from plotly.offline import iplot
from plotly.graph_objs import Scatter
py.init_notebook_mode(connected=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import igraph

## Problem 1

## Calculating and creating dataframe for data of all subjects

> We are assuming that the datasets for all subjects are stored in the **current directory of notebook**, in a folder named **FDAE Datasets** , in the format **walking_subject_1**. **Here walking represents the type of actiivity** and **subject 1 represents the subject number**.

In [5]:
datasets_location = %pwd
datasets_folder_name = "/FDAE Datasets/"

## Tasks 1.1 - 1.4 (Calculating HVG, NVG, and relevant parameters for all subjects)

> Helper function to **import data for all subjects and calculate relevant parameters**

In [6]:

# Name of relevant columns in final dataframe
columns=['Method','Subject','Accelerometer_Axis','Average_Degree','Network_Diameter','Average_Path_Length','Activity']

# Dataframe constructed from data
df = pd.DataFrame()

# Path to read files from
path = datasets_location + datasets_folder_name

def read_file(subject_num):
    # Read files
    walking_data = pd.read_csv(path+'/walking_subject{0}.csv'.format(subject_num))
    running_data = pd.read_csv(path+'/running_subject{0}.csv'.format(subject_num))
    climbingup_data = pd.read_csv(path+'/climbingup_subject{0}.csv'.format(subject_num))
    climbingdown_data = pd.read_csv(path+'/climbingdown_subject{0}.csv'.format(subject_num))
    
    # Return as tuple of files
    return walking_data, running_data, climbingup_data, climbingdown_data


def get_Graph_Values(data, axis, type_of_graph):
    
    # Build graph according to type
    if type_of_graph =='NVG':
        g = NaturalVG()
    else:
        g = HorizontalVG()
    
    # Build graph according to type and axis
    # Sampling data from 1000 - 2024
    g.build(data[axis][1000:2024])
    ig_g = g.as_igraph()
    
    # Calculate necessary values
    avg_deg, net_dia, avg_path = np.mean(ig_g.degree()), ig_g.diameter(), ig_g.average_path_length()
    
    # Return as tuple
    return avg_deg, net_dia, avg_path


def Prepare_Table(list_of_subjects):
    # Referencing the global variable declared above
    global df
    
    # Creating a list of data
    main_list = []
    for subject_num in list_of_subjects:
        
        ## Import data file
        walking_data, running_data, climbingup_data, climbingdown_data = read_file(subject_num)
        #display(walking_data)
        
        ## Activity
        for type_of_data in ['walking','running','climbing_up','climbing_down']:
            
            # Copy data for a particular subject, activity type
            if type_of_data=='walking':
                data = walking_data.copy()
            elif type_of_data=='running':
                data = running_data.copy()
            elif type_of_data=='climbing_up':
                data = climbingup_data.copy()
            elif type_of_data=='climbing_down':
                data = climbingdown_data.copy()
            
            ## axis x , y ,z
            for type_of_axis in ['x','y','z']:
                if type_of_axis == 'x':
                    axis = 'attr_x'
                elif type_of_axis == 'y':
                    axis = 'attr_y'
                else:
                    axis = 'attr_z'
                
                ## NVG or HVG
                for type_of_graph in ['NVG', 'HVG']:
                    avg_deg, net_dia, avg_path = get_Graph_Values(data, axis, type_of_graph)
                    # Collect all the created data
                    temp_list = [type_of_graph, subject_num, type_of_axis, avg_deg, net_dia, avg_path,type_of_data]
                    
                    ## append temporary list to main list
                    main_list.append(temp_list)
                    
    ## convert list of lists to dataframe
    temp_df = pd.DataFrame(main_list)
    
    # Add dataframe to main dataframe
    df = pd.concat([df, temp_df])
    
    # Correct columns
    df.columns = columns

# Subject number array
subject_number_array = np.arange(1,16)

# Create dataframe with subject number 1- 15
Prepare_Table(subject_number_array)

> After the above calculations, **data for all subjects is in one final dataframe**, Some of the data is shown below:

In [7]:
df.head()

,Method,Subject,Accelerometer_Axis,Average_Degree,Network_Diameter,Average_Path_Length,Activity
0,NVG,1,x,11.451172,10,4.441492,walking
1,HVG,1,x,3.958984,26,11.578024,walking
2,NVG,1,y,11.986328,9,4.241050,walking
3,HVG,1,y,3.951172,22,10.666252,walking
4,NVG,1,z,8.570312,10,4.897722,walking


> To ensure that the **data for all the subjects** is present:

In [8]:
df.Subject.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

> To ensure that data for **all activities** is present:

In [9]:
df.Activity.unique()

array(['walking', 'running', 'climbing_up', 'climbing_down'], dtype=object)

## Task 1.5 Plotting data - Walking and Running - Average Degree vs Network Diameter

In [10]:
# Our figure would be a 3X2 grid

fig = make_subplots(rows=3, cols=2 )

plot1_df = df[(df['Activity'].isin(['walking','running']))]

# Creating all plots

fig1 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='x')&(plot1_df['Activity']=='walking')]
fig1_1 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='x')&(plot1_df['Activity']=='running')]

fig.add_trace(go.Scatter(x=fig1['Average_Degree'], y=fig1['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=1, col=1)
fig.add_trace(go.Scatter(x=fig1_1['Average_Degree'], y=fig1_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=1, col=1)

fig2 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='x')&(plot1_df['Activity']=='walking')]
fig2_1 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='x')&(plot1_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig2['Average_Degree'], y=fig2['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(x=fig2_1['Average_Degree'], y=fig2_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=1, col=2)

fig3 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='y')&(plot1_df['Activity']=='walking')]
fig3_1 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='y')&(plot1_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig3['Average_Degree'], y=fig3['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=2, col=1)
fig.add_trace(go.Scatter(x=fig3_1['Average_Degree'], y=fig3_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=2, col=1)

fig4 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='y')&(plot1_df['Activity']=='walking')]
fig4_1 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='y')&(plot1_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig4['Average_Degree'], y=fig4['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=2, col=2)
fig.add_trace(go.Scatter(x=fig4_1['Average_Degree'], y=fig4_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=2, col=2)

fig5 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='z')&(plot1_df['Activity']=='walking')]
fig5_1 = plot1_df[(plot1_df['Method']=='NVG')&(plot1_df['Accelerometer_Axis']=='z')&(plot1_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig5['Average_Degree'], y=fig5['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=fig5_1['Average_Degree'], y=fig5_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=3, col=1)

fig6 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='z')&(plot1_df['Activity']=='walking')]
fig6_1 = plot1_df[(plot1_df['Method']=='HVG')&(plot1_df['Accelerometer_Axis']=='z')&(plot1_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig6['Average_Degree'], y=fig6['Network_Diameter'], mode="markers", marker_color='blue',name='Walking'),row=3, col=2)
fig.add_trace(go.Scatter(x=fig6_1['Average_Degree'], y=fig6_1['Network_Diameter'], mode="markers", marker_color='red',name='Running'),row=3, col=2)


# Update xaxis properties
fig.update_xaxes(title_text="Natural Visibility Graph", row=3, col=1)
fig.update_xaxes(title_text="Horizontal Visibility Graph", row=3, col=2)


# # Update yaxis properties
fig.update_yaxes(title_text="X Axis", row=1, col=1)
fig.update_yaxes(title_text="Y Axis", row=2, col=1)
fig.update_yaxes(title_text="Z Axis", row=3, col=1)

# Update layout, set title
fig.update_layout(title_text="Walking and Running - Average Degree vs Network Diameter(HVG and NVG) along X,Y and Z axis")

fig.show()

> In above figure, **across all graphs, x axis represents Average Dgree, and y axis represents Network Diameter** .

## Task 1.6 Plotting data - Climbing Up and Climbing down - Average Degree vs Network Diameter

In [11]:
fig = make_subplots(rows=3, cols=2, )

plot2_df = df[(df['Activity'].isin(['climbing_up','climbing_down']))]

fig1 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='x')&(plot2_df['Activity']=='climbing_up')]
fig1_1 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='x')&(plot2_df['Activity']=='climbing_down')]

fig.add_trace(go.Scatter(x=fig1['Average_Degree'], y=fig1['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=1, col=1)
fig.add_trace(go.Scatter(x=fig1_1['Average_Degree'], y=fig1_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=1, col=1)

fig2 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='x')&(plot2_df['Activity']=='climbing_up')]
fig2_1 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='x')&(plot2_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig2['Average_Degree'], y=fig2['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(x=fig2_1['Average_Degree'], y=fig2_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=1, col=2)

fig3 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='y')&(plot2_df['Activity']=='climbing_up')]
fig3_1 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='y')&(plot2_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig3['Average_Degree'], y=fig3['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=2, col=1)
fig.add_trace(go.Scatter(x=fig3_1['Average_Degree'], y=fig3_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=2, col=1)

fig4 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='y')&(plot2_df['Activity']=='climbing_up')]
fig4_1 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='y')&(plot2_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig4['Average_Degree'], y=fig4['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=2, col=2)
fig.add_trace(go.Scatter(x=fig4_1['Average_Degree'], y=fig4_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=2, col=2)

fig5 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='z')&(plot2_df['Activity']=='climbing_up')]
fig5_1 = plot2_df[(plot2_df['Method']=='NVG')&(plot2_df['Accelerometer_Axis']=='z')&(plot2_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig5['Average_Degree'], y=fig5['Network_Diameter'], mode="markers", marker_color='blue', showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=fig5_1['Average_Degree'], y=fig5_1['Network_Diameter'], mode="markers", marker_color='red', showlegend=False),row=3, col=1)

fig6 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='z')&(plot2_df['Activity']=='climbing_up')]
fig6_1 = plot2_df[(plot2_df['Method']=='HVG')&(plot2_df['Accelerometer_Axis']=='z')&(plot2_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig6['Average_Degree'], y=fig6['Network_Diameter'], mode="markers", marker_color='blue',name='Climbing Up'),row=3, col=2)
fig.add_trace(go.Scatter(x=fig6_1['Average_Degree'], y=fig6_1['Network_Diameter'], mode="markers", marker_color='red',name='Climbing Down'),row=3, col=2)


# Update xaxis properties
fig.update_xaxes(title_text="Natural Visibility Graph", row=3, col=1)
fig.update_xaxes(title_text="Horizontal Visibility Graph", row=3, col=2)


# # Update yaxis properties
fig.update_yaxes(title_text="X Axis", row=1, col=1)
fig.update_yaxes(title_text="Y Axis", row=2, col=1)
fig.update_yaxes(title_text="Z Axis", row=3, col=1)

# Update layout, set title
fig.update_layout(title_text="Climbing up and Climbing down - Average Degree vs Network Diameter(HVG and NVG) along X,Y and Z axis")

fig.show()

> In above figure, **across all graphs, x axis represents Average Dgree, and y axis represents Network Diameter**.

## Problem 2

###  Helper functions to calculate ordinal patterns, entropy and complexity

In [12]:
def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector representing the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    permutation = np.argsort(partition)
    idx = _hash(permutation)

    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = (idx == i).sum()
    return list(counts[counts != 0].astype(int))

def _hash(x):
    m, n = x.shape
    if n == 1:
        return np.zeros(m)
    return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:]) 
    

def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

## Task 2.1 - 2.2 Calculating permutation entropy and complexity for all subjects by varying dimension, delay and  signal length across different axis

> Below function **calculates entropy and complexity for all subjects and activities with varying values of delay, dimension and signal length**.

In [13]:
columns=['Subject number','Accelerometer_Axis','Signal_Length','Dimension','Delay','Permutation_Entropy', 'Complexity','Activity']
df = pd.DataFrame()

path = datasets_location + datasets_folder_name

# Reading files helper function
def read_file(subject_num):
    walking_data = pd.read_csv(path+'/walking_subject{0}.csv'.format(subject_num))
    running_data = pd.read_csv(path+'/running_subject{0}.csv'.format(subject_num))
    climbingup_data = pd.read_csv(path+'/climbingup_subject{0}.csv'.format(subject_num))
    climbingdown_data = pd.read_csv(path+'/climbingdown_subject{0}.csv'.format(subject_num))

    return walking_data, running_data, climbingup_data, climbingdown_data

# Calculate entropy and complexity
def getEntropy(data, dim, delay):
    op = ordinal_patterns(data, dim, delay)
    return p_entropy(op), complexity(op)

def Prepare_Table(list_of_subjects):
    global df
    main_list = []
    for subject_num in list_of_subjects:
        
        ## Import data file
        walking_data, running_data, climbingup_data, climbingdown_data = read_file(subject_num)
        ## display(walking_data)
        
        ## Activity
        for type_of_data in ['walking','running','climbing_up','climbing_down']:
            if type_of_data=='walking':
                data = walking_data.copy()
            elif type_of_data=='running':
                data = running_data.copy()
            elif type_of_data=='climbing_up':
                data = climbingup_data.copy()
            elif type_of_data=='climbing_down':
                data = climbingdown_data.copy()
            
            ## axis
            for type_of_axis in ['x','y','z']:
                if type_of_axis == 'x':
                    axis = 'attr_x'
                elif type_of_axis == 'y':
                    axis = 'attr_y'
                else:
                    axis = 'attr_z'
                
                ## Signal length
                for sig_len in [1024, 2048, 4096]:
                    ## Dimension
                    for dim in [3,4,5,6]:
                        ## Delay
                        for delay in [1,2,3]:
                            entropy, complexity = getEntropy(data[axis][0:sig_len],dim, delay)
                    
                            
                            temp_list = [subject_num, type_of_axis, sig_len, dim, delay, entropy, complexity, type_of_data]
                            ## append elementary list to main list
                            main_list.append(temp_list)
    ## convert list of lists to dataframe
    temp_df = pd.DataFrame(main_list)
    df = pd.concat([df, temp_df])
    df.columns = columns
    
subject_number_array = np.arange(1,16)
Prepare_Table(subject_number_array)

> After the above calculations, **data for all subjects is in one final dataframe**, Some of the data is shown below:

In [14]:
df.head()

,Subject number,Accelerometer_Axis,Signal_Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,1,x,1024,3,1,0.855987,0.119191,walking
1,1,x,1024,3,2,0.960568,0.036283,walking
2,1,x,1024,3,3,0.982132,0.016977,walking
3,1,x,1024,4,1,0.794190,0.199622,walking
4,1,x,1024,4,2,0.926945,0.085968,walking


> To ensure that the **data for all the subjects** is present:

In [15]:
df["Subject number"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

> To ensure that all **activity types data** is present

In [16]:
df["Activity"].unique()

array(['walking', 'running', 'climbing_up', 'climbing_down'], dtype=object)

> To ensure that **dimension, delay and signal length** are varied correctly

In [17]:
df.Signal_Length.unique()

array([1024, 2048, 4096], dtype=int64)

In [18]:
df.Dimension.unique()

array([3, 4, 5, 6], dtype=int64)

In [19]:
df.Delay.unique()

array([1, 2, 3], dtype=int64)

## Task 2.3  Plotting Permutation entropy vs Complexity for walking and running activity in subjects. (signal length =4096, embedded delay = 1 along data of axis x,y and z)

In [20]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=4)

# Signal length = 4096 and delay = 1
# Activities running and walking
plot3_df = df[(df['Signal_Length']==4096)&(df['Delay']==1)&(df['Activity'].isin(['walking','running']))]


fig1 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='walking')]
fig1_1 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig1['Permutation_Entropy'], y=fig1['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=1)
fig.add_trace(go.Scatter(x=fig1_1['Permutation_Entropy'], y=fig1_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=1)

fig2 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='walking')]
fig2_1 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig2['Permutation_Entropy'], y=fig2['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(x=fig2_1['Permutation_Entropy'], y=fig2_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=2)

fig3 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='walking')]
fig3_1 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig3['Permutation_Entropy'], y=fig3['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=3)
fig.add_trace(go.Scatter(x=fig3_1['Permutation_Entropy'], y=fig3_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=3)

fig4 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='walking')]
fig4_1 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='x')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig4['Permutation_Entropy'], y=fig4['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=4)
fig.add_trace(go.Scatter(x=fig4_1['Permutation_Entropy'], y=fig4_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=4)


fig5 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='walking')]
fig5_1 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig5['Permutation_Entropy'], y=fig5['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=1)
fig.add_trace(go.Scatter(x=fig5_1['Permutation_Entropy'], y=fig5_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=1)

fig6 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='walking')]
fig6_1 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig6['Permutation_Entropy'], y=fig6['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=2)
fig.add_trace(go.Scatter(x=fig6_1['Permutation_Entropy'], y=fig6_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=2)

fig7 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='walking')]
fig7_1 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig7['Permutation_Entropy'], y=fig7['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=3)
fig.add_trace(go.Scatter(x=fig7_1['Permutation_Entropy'], y=fig7_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=3)

fig8 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='walking')]
fig8_1 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='y')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig8['Permutation_Entropy'], y=fig8['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=4)
fig.add_trace(go.Scatter(x=fig8_1['Permutation_Entropy'], y=fig8_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=4)



fig9 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='walking')]
fig9_1 = plot3_df[(plot3_df['Dimension']==3)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig9['Permutation_Entropy'], y=fig9['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=fig9_1['Permutation_Entropy'], y=fig9_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=1)

fig10 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='walking')]
fig10_1 = plot3_df[(plot3_df['Dimension']==4)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig10['Permutation_Entropy'], y=fig10['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=2)
fig.add_trace(go.Scatter(x=fig10_1['Permutation_Entropy'], y=fig10_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=2)

fig11 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='walking')]
fig11_1 = plot3_df[(plot3_df['Dimension']==5)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig11['Permutation_Entropy'], y=fig11['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=3)
fig.add_trace(go.Scatter(x=fig11_1['Permutation_Entropy'], y=fig11_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=3)

fig12 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='walking')]
fig12_1 = plot3_df[(plot3_df['Dimension']==6)&(plot3_df['Accelerometer_Axis']=='z')&(plot3_df['Activity']=='running')]
fig.add_trace(go.Scatter(x=fig12['Permutation_Entropy'], y=fig12['Complexity'], mode="markers", marker_color='blue', name='Walking'),row=3, col=4)
fig.add_trace(go.Scatter(x=fig12_1['Permutation_Entropy'], y=fig12_1['Complexity'], mode="markers", marker_color='red', name='Running'),row=3, col=4)


# Update xaxis properties
fig.update_xaxes(title_text="Dimension 3", row=3, col=1)
fig.update_xaxes(title_text="Dimension 4", row=3, col=2)
fig.update_xaxes(title_text="Dimension 5", row=3, col=3)
fig.update_xaxes(title_text="Dimension 6", row=3, col=4)

# # Update yaxis properties
fig.update_yaxes(title_text="X Axis", row=1, col=1)
fig.update_yaxes(title_text="Y Axis", row=2, col=1)
fig.update_yaxes(title_text="Z Axis", row=3, col=1)

fig.update_layout(title_text = "Permutation entropy (x axis) vs Complexity (y axis) for - Walking and Running activities (signal length =4096, embedded delay = 1)")

fig.show()

> In above figure, across all graphs, **x axis represents Permutation Entropy, and y axis represents Complexity** .

## Task 2.4  Plotting Permutation entropy vs Complexity for Climbing up and Climbing down activity in subjects. (signal length =4096, embedded delay = 1 along data of axis x,y and z)

In [21]:
fig = make_subplots(rows=3, cols=4, )

# Signal length = 4096 and delay = 1
# Activities climbing up and climbing down
plot4_df = df[(df['Signal_Length']==4096)&(df['Delay']==1)&(df['Activity'].isin(['climbing_up','climbing_down']))]


fig1 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_up')]
fig1_1 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig1['Permutation_Entropy'], y=fig1['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=1)
fig.add_trace(go.Scatter(x=fig1_1['Permutation_Entropy'], y=fig1_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=1)

fig2 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_up')]
fig2_1 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig2['Permutation_Entropy'], y=fig2['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(x=fig2_1['Permutation_Entropy'], y=fig2_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=2)

fig3 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_up')]
fig3_1 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig3['Permutation_Entropy'], y=fig3['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=3)
fig.add_trace(go.Scatter(x=fig3_1['Permutation_Entropy'], y=fig3_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=3)

fig4 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_up')]
fig4_1 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='x')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig4['Permutation_Entropy'], y=fig4['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=1, col=4)
fig.add_trace(go.Scatter(x=fig4_1['Permutation_Entropy'], y=fig4_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=1, col=4)


fig5 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_up')]
fig5_1 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig5['Permutation_Entropy'], y=fig5['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=1)
fig.add_trace(go.Scatter(x=fig5_1['Permutation_Entropy'], y=fig5_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=1)

fig6 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_up')]
fig6_1 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig6['Permutation_Entropy'], y=fig6['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=2)
fig.add_trace(go.Scatter(x=fig6_1['Permutation_Entropy'], y=fig6_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=2)

fig7 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_up')]
fig7_1 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig7['Permutation_Entropy'], y=fig7['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=3)
fig.add_trace(go.Scatter(x=fig7_1['Permutation_Entropy'], y=fig7_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=3)

fig8 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_up')]
fig8_1 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='y')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig8['Permutation_Entropy'], y=fig8['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=2, col=4)
fig.add_trace(go.Scatter(x=fig8_1['Permutation_Entropy'], y=fig8_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=2, col=4)



fig9 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_up')]
fig9_1 = plot4_df[(plot4_df['Dimension']==3)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig9['Permutation_Entropy'], y=fig9['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=fig9_1['Permutation_Entropy'], y=fig9_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=1)

fig10 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_up')]
fig10_1 = plot4_df[(plot4_df['Dimension']==4)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig10['Permutation_Entropy'], y=fig10['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=2)
fig.add_trace(go.Scatter(x=fig10_1['Permutation_Entropy'], y=fig10_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=2)

fig11 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_up')]
fig11_1 = plot4_df[(plot4_df['Dimension']==5)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig11['Permutation_Entropy'], y=fig11['Complexity'], mode="markers", marker_color='blue', showlegend=False),row=3, col=3)
fig.add_trace(go.Scatter(x=fig11_1['Permutation_Entropy'], y=fig11_1['Complexity'], mode="markers", marker_color='red', showlegend=False),row=3, col=3)

fig12 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_up')]
fig12_1 = plot4_df[(plot4_df['Dimension']==6)&(plot4_df['Accelerometer_Axis']=='z')&(plot4_df['Activity']=='climbing_down')]
fig.add_trace(go.Scatter(x=fig12['Permutation_Entropy'], y=fig12['Complexity'], mode="markers", marker_color='blue', name='Climbing Up'),row=3, col=4)
fig.add_trace(go.Scatter(x=fig12_1['Permutation_Entropy'], y=fig12_1['Complexity'], mode="markers", marker_color='red', name='Climbing Down'),row=3, col=4)


# Update xaxis properties
fig.update_xaxes(title_text="Dimension 3", row=3, col=1)
fig.update_xaxes(title_text="Dimension 4", row=3, col=2)
fig.update_xaxes(title_text="Dimension 5", row=3, col=3)
fig.update_xaxes(title_text="Dimension 6", row=3, col=4)

# # Update yaxis properties
fig.update_yaxes(title_text="X Axis", row=1, col=1)
fig.update_yaxes(title_text="Y Axis", row=2, col=1)
fig.update_yaxes(title_text="Z Axis", row=3, col=1)

fig.update_layout(title_text = "Permutation entropy (x axis) vs Complexity (y axis) for - Climbing up and Climbing down activities (signal length =4096, embedded delay = 1)")


fig.show()

> In above figure, across all graphs, **x axis represents Permutation Entropy, and y axis represents Complexity** .